# Table 2
This file produces the results for "Table 2. Fruitvale Employment Profile".  

In [1]:
#Import packages
import pandas as pd
import matplotlib as mpl
import numpy as np
import math
import os
pd.options.mode.chained_assignment = None #gets rid of warning for chained alterations

In [2]:
#set directories
os.chdir("/Users/briangoggin/Dropbox/CP 201A/Fruitvale")
cwd = os.getcwd()
root = cwd+"/Raw Data/" #root is directory to raw files
root

'/Users/briangoggin/Dropbox/CP 201A/Fruitvale/Raw Data/'

In [3]:
#set input data source
employment = root+"ACS_14_5YR_B23025_with_ann.csv"


In [4]:
#identify Fruitvale Census Tracts
Fruitvale = [6001406100, 6001406201, 6001406202, 6001406300, 6001406500, 6001407101, 6001407102, 6001407200, 6001407300, 6001407400, 6001407500, 6001407600]


In [5]:
#Define function for import and standard ACS cleaning operations
def import_census(file):
    df = pd.read_csv(file, header = 1)
    df['Fruitvale'] = df['Id2'].isin(Fruitvale)
    #the following 5 lines create duplicate observations of Fruitvale to include them in county averages
    df2 = df[df['Fruitvale'] == True]
    df2.drop('Fruitvale', axis = 1)
    df2.is_copy = False
    df2['Fruitvale'] = False
    df = df.append(df2)
    return df

#Define function for proportions MOE
def prop_MOE(numerator, denominator, num_moe, den_moe):
    return ((num_moe**2 - (((numerator/denominator)**2)*(den_moe**2))))**(1/2)/denominator

In [6]:
#import data
employment = import_census(employment)

In [7]:
#Convert all employment variables to integers
employment['pop'] = employment["Estimate; Total:"].astype(int)
employment['pop_moe'] = employment["Margin of Error; Total:"].astype(int)
employment['in_labor'] = employment['Estimate; In labor force:'].astype(int)
employment['in_labor_moe'] = employment['Margin of Error; In labor force:'].astype(int)
employment['civ_employed'] = employment["Estimate; In labor force: - Civilian labor force: - Employed"].astype(int)
employment['civ_employed_moe'] = employment["Margin of Error; In labor force: - Civilian labor force: - Employed"].astype(int)
employment['armed'] = employment["Estimate; In labor force: - Armed Forces"].astype(int)
employment['armed_moe'] = employment["Margin of Error; In labor force: - Armed Forces"].astype(int)
employment['unemployed'] = employment["Estimate; In labor force: - Civilian labor force: - Unemployed"].astype(int)
employment['unemployed_moe'] = employment["Margin of Error; In labor force: - Civilian labor force: - Unemployed"].astype(int)
employment['not_in_labor'] = employment["Estimate; Not in labor force"].astype(int)
employment['not_in_labor_moe'] = employment["Margin of Error; Not in labor force"].astype(int)


In [8]:
#Create employment standard MOEs

moes = ['pop_moe', 'in_labor_moe', 'civ_employed_moe', 'armed_moe', 
               'unemployed_moe', 'not_in_labor_moe']

#convert MOEs to 95% confidence level
def convert(column):
    return column*(1.96/1.645)

#square columns
def square(column):
    return column**2

for item in moes:
    employment[item] = convert(employment[item])
    employment[item+'2']= square(employment[item])

#create 'total empoyed' category, its moe, and its moe squared
employment['total_employed'] = employment['civ_employed'] + employment['armed']
employment['total_employed_moe'] = (employment['civ_employed_moe2'] + employment['armed_moe2'])**(1/2) 
employment['total_employed_moe2'] = (employment['total_employed_moe'])**2


In [9]:
#list data to include in sum
employments = ['pop', 'total_employed', 'unemployed', 'not_in_labor', 'in_labor', 
            'pop_moe2', 'total_employed_moe2', 'unemployed_moe2', 'not_in_labor_moe2', 'in_labor_moe2']


In [10]:
#Sum data 
exemployment = employment.groupby('Fruitvale')[employments].sum()

#new MOEs
exemployment['pop_moe'] = (exemployment['pop_moe2'])**(1/2)
exemployment['total_employed_moe'] = (exemployment['total_employed_moe2'])**(1/2)
exemployment['unemployed_moe'] = (exemployment['unemployed_moe2'])**(1/2)
exemployment['not_in_labor_moe'] = (exemployment['not_in_labor_moe2'])**(1/2)
exemployment['in_labor_moe'] = (exemployment['in_labor_moe2'])**(1/2)

#percentages
exemployment['pct_employed'] = 100*(exemployment['total_employed']/exemployment['pop'])
exemployment['pct_unemployed'] = 100*(exemployment['unemployed']/exemployment['pop'])
exemployment['pct_not_in_labor'] = 100*(exemployment['not_in_labor']/exemployment['pop'])
exemployment['pct_in_labor'] = 100*(exemployment['in_labor']/exemployment['pop'])

In [11]:
#generate proportion MOEs
exemployment['pct_employed_moe'] = 100*(prop_MOE(exemployment['total_employed'], exemployment['pop'], exemployment['total_employed_moe'], exemployment['pop_moe']))
exemployment['pct_unemployed_moe'] = 100*(prop_MOE(exemployment['unemployed'], exemployment['pop'], exemployment['unemployed_moe'], exemployment['pop_moe']))
exemployment['pct_not_in_labor_moe'] = 100*(prop_MOE(exemployment['not_in_labor'], exemployment['pop'], exemployment['not_in_labor_moe'], exemployment['pop_moe']))
exemployment['pct_in_labor_moe'] = 100*(prop_MOE(exemployment['in_labor'], exemployment['pop'], exemployment['in_labor_moe'], exemployment['pop_moe']))


In [12]:
#set variables to keep 
keep_vars = ['pop', 'total_employed', 'unemployed', 'not_in_labor', 
             'pop_moe', 'total_employed_moe', 'unemployed_moe', 'not_in_labor_moe', 
            'pct_employed', 'pct_unemployed', 'pct_not_in_labor', 
            'pct_employed_moe', 'pct_unemployed_moe', 'pct_not_in_labor_moe', 'in_labor',
            'in_labor_moe', 'pct_in_labor', 'pct_in_labor_moe']

exemployment = exemployment[keep_vars]
exemployment = exemployment.round(2)

In [13]:
#export data to csv
export_path = cwd+'/Output/python_output/employment.csv'
exemployment = exemployment.to_csv(export_path)